<a href="https://colab.research.google.com/github/dxmai/CS114.L21.KHCL/blob/main/FinalProject/FinalProject_FakeNews_CollectData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import thư viện

In [ ]:
import pandas as pd
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Hàm lọc lại năm

In [ ]:
def Filter_Year(dataFrame):
  yearsList = ['2021', '2020', '2019', '2018']
  delete_list = []
  for row in range(len(dataFrame)):
    flag = False
    for year in yearsList:
        if year in dataFrame.iloc[row]['date']:
          flag = True
          break
    if not flag:
      delete_list.append(row)
  return delete_list

In [ ]:
def Delete_From_DataFrame(dataFrame):
  delete_list = Filter_Year(dataFrame)
  for index in delete_list:
     dataFrame.drop(index=index, inplace=True)
  dataFrame.reset_index(drop=True, inplace=True)

#Hàm lọc lại những giá trị bị trùng, none,...

In [ ]:
def Filter(dataFrame):
  dataFrame.dropna(inplace=True)
  dataFrame.drop(dataFrame[dataFrame['date'] == ""].index, inplace=True)
  dataFrame.drop(dataFrame[dataFrame['title'] == ""].index, inplace=True)
  dataFrame.drop(dataFrame[dataFrame['text'] == ""].index, inplace=True)
  dataFrame.drop_duplicates(inplace=True)
  dataFrame.reset_index(drop=True, inplace=True)

#Trang tin giả

##Activistpost

In [ ]:
import scrapy

class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://www.activistpost.com/'
        ]
        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        getLinkInDiv = response.css('h3.content-list-title')
        allLinks = getLinkInDiv.css('a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = response.css('a.next::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)


    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018'] 
        available = False 
        date = response.css('.entry-meta span::text').get() 
        for year in yearsList: 
            if year in date: 
                available = True 
                break
        if available: 
            title = response.css('.entry-title::text').get()
            title = title.replace('\n', '')
            title = title.replace('\t', '')
            paragraph = response.css('.entry-content p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 4:
                    break
                if count == 0:
                    count += 1
                    continue
                sentence = sentence.replace('\xa0', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 1
            }

In [ ]:
activist = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/activistpost.json')
Filter(activist)
activist

,date,title,text,is_fake
0,2021-05-28,US Mint Delays Silver Shipments Due To “Global...,Interest in silver is soaring (both for indust...,1
1,2021-06-28,"Research Paper Exposes Cybersecurity, Environm...",A 2018 survey revealed did NOT want to live i...,1
2,2021-05-29,DeSantis’s Anti-Riot Law Undermines Two Import...,When Florida Gov. Ron DeSantis spent the last ...,1
3,2021-05-28,Another Massive Cargo Ship Was Just Stuck In t...,To quote the great Los Angeles sportscaster Vi...,1
4,2021-06-25,BofA Crashes The “Transitory” Party: Sees Up T...,"At the start of May, when observing the avalan...",1
...,...,...,...,...
10253,2019-12-06,Comedian Fined $35k for Offensive Joke (And Ot...,Are you ready for this week’s absurdity? Here’...,1
10254,2019-12-06,Bill Gates Wants to Export India’s National ID...,It’s not just a social credit score system spr...,1
10255,2019-12-05,Pro Tip: Mentally Replace All Uses Of “Conspir...,The corrupt mechanisms which gave rise to the ...,1
10256,2019-12-05,"Despite Expert Warnings and Accidents, Elected...","According to many experts, Automated Vehicles ...",1


##Natural News

In [ ]:
import scrapy


class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://www.naturalnews.com/all-posts.html'
        ]
        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        getLinkInDiv = response.css('div.f-tabbed-list-content')
        allLinks = getLinkInDiv.css('h2 a::attr(href)').getall()

        for each_link in allLinks:
            each_link = 'https://www.naturalnews.com/' + each_link
            yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = response.css('.pagination-next a::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']
        available = False
        date = response.css('.Article-Author::text').get()
        date = date.replace(' by: ', '')
        for year in yearsList:
            if year in date:
                available = True
                break
        if available:
            title = response.css('.entry-title::text').get()
            title = title.replace('\n', '')
            title = title.replace('\t', '')
            paragraph = response.css('.entry-content p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 4:
                    break
                sentence = sentence.replace('\xa0', '')
                sentence = sentence.replace('\n', '')
                sentence = sentence.replace('\t', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 1
            }

In [ ]:
natural = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/naturalnews.json')
Filter(natural)
natural

,date,title,text,is_fake
0,2021-06-29,Honda announces plan to design and manufacture...,) Japanese automobile manufacturer Honda Motor...,1
1,2021-06-29,Russia calls America a ‘liberal totalitarian s...,) Russian authoritiesof the failing U.S. regim...,1
2,2021-06-29,"In Joe Biden’s America, Whites are the enemy",) Whatthose silly white people so nervous abou...,1
3,2021-06-29,Heart health and drug safety: Common cold medi...,) The common cold is caused by a viral infecti...,1
4,2021-06-29,Garbage-gobbling machine combats marine pollut...,) Afloating in the water has been cleaning up ...,1
...,...,...,...,...
11980,2020-03-03,NHS: Healthcare facilities across the UK will ...,) As the world scrambles to prepare and combat...,1
11981,2020-03-03,World Bank “pandemic bonds” may explain why th...,) Even as the first American dies of the Wuhan...,1
11982,2020-03-03,Estimated 300 – 500 coronavirus cases already ...,) One of the more fascinating things to note i...,1
11983,2020-03-04,One mistake is all it takes: 6 Dangerous survi...,) There are many beliefs about survival and pr...,1


##Zerohedge

In [ ]:
import scrapy
dateValid = True

class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://www.zerohedge.com/'
        ]
        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        getLinkInDiv = response.css('div.Article_mobileNonSticky__PmGNH')
        allLinks = getLinkInDiv.css('h2 a::attr(href)').getall()

        for each_link in allLinks:
            if dateValid:
                each_link = 'https://www.zerohedge.com' + each_link
                yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = response.css('a.SimplePaginator_next__15okP::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']
        available = False
        global dateValid
        date = response.css('.ArticleFull_headerFooter__date__3T7FN::text').get()
        for year in yearsList:
            if year in date:
                available = True
                break
        if available:
            title = response.css('.ArticleFull_title__2cUI6::text').get()
            title = title.replace('\n', '')
            title = title.replace('\t', '')
            paragraph = response.css('.NodeContent_body__2clki p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 4:
                    break
                sentence = sentence.replace('\xa0', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 1
            }
        else:
            dateValid = False

In [ ]:
zerohedge = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/zerohedge.json')
Filter(natural)
zerohedge

,date,title,text,is_fake
0,2021-07-01 09:05:00,Complacent Goldilocks Got Eaten By Bear,"This morning I just watched a massive, fully l...",1
1,2021-07-01 09:28:00,New Video Shows Surfside Condo's Parking Garag...,Video recorded moments before the Champlain To...,1
2,2021-07-01 08:35:00,Almost 15 Million Americans Remain On Governme...,With more states ending their emergency handou...,1
3,2021-07-01 08:45:00,Nio Shares Pop 3% Pre-Market After Company Buc...,Shares of EV automaker NIO are up about 3% in ...,1
4,2021-07-01 07:15:00,Trump Organization CFO Alan Weisselberg Surren...,Weisselberg has pleaded not guilty to the char...,1
...,...,...,...,...
366,2021-06-30 20:05:00,Charges Filed Against Trump Org And CFO Alan W...,A Manhattan grand jury has filed the anticipat...,1
367,2021-07-01 12:26:00,Rescue Operations Halted At Collapsed Surfside...,One week after the Champlain Towers South buil...,1
368,2021-07-01 12:30:00,Ireland One Of 9 Holdouts Who Refused To Sign ...,"In a rare scoop, the Irish Times just reporte...",1
369,2021-07-01 06:44:00,Kamala Harris Staffers Are Leaking -- And Her ...,Vice President Kamala Harris' office is a toxi...,1


##Prntly

In [ ]:
import scrapy
dateValid = True

class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://prntly.com/category/science/',
            'https://prntly.com/category/politics/',
            'https://prntly.com/category/world/europe/',
            'https://prntly.com/category/world/asia/',
            'https://prntly.com/category/world/middle-east/',
            'https://prntly.com/category/local-news/',
            'https://prntly.com/category/trade-jobs/',
            'https://prntly.com/category/immigration/'
        ]
        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        getLinkInDiv = response.css('div.entry-content-holder')
        allLinks = getLinkInDiv.css('h2 a::attr(href)').getall()

        for each_link in allLinks:
            if dateValid:
                yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = response.css('a.next::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']
        available = False
        global dateValid
        date = response.css('.entry-date::text').get()
        for year in yearsList:
            if year in date:
                available = True
                break
        if available:
            title = response.css('h1.entry-title::text').get()
            title = title.replace('\n', '')
            title = title.replace('\t', '')
            paragraph = response.css('.entry-content p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 4:
                    break
                sentence = sentence.replace('\xa0', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 1
            }
        else:
            dateValid = False


In [ ]:
prntly = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/prntly.json')
Filter(prntly)
prntly

,date,title,text,is_fake
0,2020-01-28,Woman With C-Virus BRAGS Online About Sneaking...,A chinese national from the quarantined city o...,1
1,2020-02-07,Death of the first whistleblower of the Corona...,"The death of Dr. Li Wenliang has spark anger, ...",1
2,2020-02-28,USA Leads The World In Coronavirus Recovery At...,"\nAcross the globe, the media spreads panic ab...",1
3,2020-02-28,BREAKING: Coronavirus Infected SPIKE In South ...,Seoul- over 2000 people have now come down wit...,1
4,2020-02-06,Nancy Pelosi Is Becoming Unhinged Before Our V...,\n\nTrashy Nancy disgusted the nation last nig...,1
...,...,...,...,...
561,2020-01-13,Dem attempting to insult Trump’s Farsi tweet m...,A democrat from Florida learned the hard way t...,1
562,2020-01-13,Breaking News: United Kingdom Claims Iran Brie...,In the midst of street protests on the verge o...,1
563,2020-01-13,president Trump’s Farsi tweet to iranian prote...,"President Trump made Iranian history, but not ...",1
564,2020-01-13,"As Iranians protest against the regime, Americ...",Confusion in the world of politics this week. ...,1


##Thegateway

In [ ]:
import scrapy
dateValid = True

class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://www.thegatewaypundit.com/'
        ]
        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        if dateValid:
            getLinkInDiv = response.css('div.tgp-post')
            allLinks = getLinkInDiv.css('.entry-archive-title a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = response.css('.next a::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']
        available = False
        global dateValid
        date = response.css('.entry-meta-text span::text')[2].get()
        date = date.replace('\nPublished ', '')
        temp = date.split(' at', 1)
        date = temp[0]
        for year in yearsList:
            if year in date:
                available = True
                break
        if available:
            title = response.css('h1.entry-title::text').get()
            title = title.replace('\n', '')
            title = title.replace('\t', '')
            paragraph = response.css('.entry-content p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 3:
                    break
                sentence = sentence.replace('\xa0', '')
                sentence = sentence.replace('\n', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 1
            }
        else:
            dateValid = False


In [ ]:
thegateway = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/thegateway.json')
Filter(thegateway)
thegateway

,date,title,text,is_fake
0,"July 1, 2021",Man Tackles Woman to the Ground and Sexually A...,Violent crime and murder are skyrocketing in N...,1
1,"July 1, 2021",FIREWORKS: Radical Far-Left Media Voices From ...,Members of various media outlets attacked one ...,1
2,"July 1, 2021",Today the Chinese Communist Party Celebrates 1...,Check it out .Advertisement - story continues ...,1
3,"July 1, 2021",Discrimination Lawsuit Filed Against Chicago-A...,The Southwest Legal Foundation alleges violat...,1
4,"July 1, 2021",Hit Me Baby One More Time – Judge Shoots Down ...,Advertisement - story continues below:A judge ...,1
...,...,...,...,...
36649,"June 30, 2021",Report: Ashli Babbitt Shooter Is Member of Mik...,"Display at funeral service for Ashli Babbitt, ...",1
36650,"June 30, 2021","“If You Think I’m Kidding, I’m Not” – Joe Bide...",Joe Biden on Wednesday called Congresswoman Ma...,1
36651,"June 30, 2021",BREAKING: New York DOJ to Indict Trump CFO and...,President Trump speaks at Save America rally i...,1
36652,"June 30, 2021",“Yes” – Trump When Asked if He Has Made Up His...,President Trump sat down with Fox News host Se...,1


##Conservativedailypost 

In [ ]:
import scrapy
dateValid = True


class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://conservativedailypost.com/category/politics/'
            'https://conservativedailypost.com/category/u-s/',
            'https://conservativedailypost.com/category/world/'
        ]
        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        if dateValid:
            getLinkInDiv = response.css('div.mob')
            allLinks = getLinkInDiv.css('h4 a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = response.css('a.next::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']
        available = False
        global dateValid
        date = response.css('.date-published::text')[-1].get()
        date = date.replace('\n', '')
        date = date.replace('\t', '')
        date = date.replace('\r', '')
        for year in yearsList:
            if year in date:
                available = True
                break
        if available:
            title = response.css('h1.min-title::text').get()
            title = title.replace('\n', '')
            title = title.replace('\t', '')
            paragraph = response.css('.content-container p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 3:
                    break
                sentence = sentence.replace('\xa0', '')
                sentence = sentence.replace('\n', '')
                sentence = sentence.replace('\r', '')
                sentence = sentence.replace('\t', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 1
            }
        else:
            dateValid = False

In [ ]:
conservative = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/conservative.json')
Filter(conservative)
conservative

,date,title,text,is_fake
0,2021-07-01 13:00:00,"\rUSB Flash Drives Stolen, Transferred, Used I...",In multiple swing states flash drives (USBs) u...,1
1,2021-07-01 17:54:00,\rPA Supreme Court Overturns Cosby Conviction,The supreme court of Pennsylvania has found th...,1
2,2021-07-01 18:18:00,\rDelta Variant Being Used For Next Round Of L...,CNN rolled out Dr. Peter Hotez of Houston to s...,1
3,2021-07-01 19:01:00,\rNavy Forces Unit To March With ‘Gay’ US Flag...,Active duty members of the in San Diego were ...,1
4,2021-07-01 16:41:00,"\rFauci Announces Two Americas…. One Vax, One ...",In an appearance on Dom Lemon’s CNN panic hour...,1
...,...,...,...,...
4883,2018-01-23 00:29:00,\rChina Weighs In On “Superior” Action As Comm...,Just as the Soviet Union made during the previ...,1
4884,2018-01-23 00:23:00,\rArabs Ejected: Pence Rocks Chamber As “Your ...,U.S. Vice President Mike Pence told Israel’s K...,1
4885,2018-01-23 00:16:00,\r“Addicted To The Attention”: Serial Offender...,Judge William Raines is done feeling sorry. “I...,1
4886,2018-01-22 23:45:00,\rBank Calls 911 As Man Issues “Differing From...,Police report on an unusual situation involvin...,1


##Dailyheadlines

In [ ]:
import scrapy
dateValid = True


class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'http://dailyheadlines.com/'
        ]
        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        if dateValid:
            getLinkInDiv = response.css('h2.entry-title')
            allLinks = getLinkInDiv.css('a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = response.css('.pagination a::attr(href)')[-1].get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']
        available = False
        global dateValid
        date = response.css('.post-meta-date::text')[-1].get()
        date = date.replace('\n', '')
        date = date.replace(' ' * 21, '')
        print(date)
        for year in yearsList:
            if year in date:
                available = True
                break
        if available:
            title = response.css('h2.post-title::text').get()
            title = title.replace('\n', '')
            title = title.replace('\t', '')
            title = title.replace(' ' * 12, '')
            title = title.replace(' ' * 9, '')
            paragraph = response.css('.entry-content p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 3:
                    break
                sentence = sentence.replace('\xa0', '')
                sentence = sentence.replace('\n', '')
                sentence = sentence.replace('\r', '')
                sentence = sentence.replace('\t', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 1
            }
        else:
            dateValid = False



In [ ]:
dailyheadline = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/dailyheadline.json')
Filter(dailyheadline)
dailyheadline

,date,title,text,is_fake
0,2021-06-25,73-Year-Old Pastor and Purple Heart Veteran Ar...,The witchhunt from the January 6 so-called “ri...,1
1,2021-06-25,Leftist Thug Reporters Threatening Anyone Asso...,“They’re coming for me because I fight for the...,1
2,2021-06-24,ALERT: Kamala Now Telling People To Harass The...,Biden’s July 4th vaccine goal is falling short...,1
3,2021-07-01,JAWDROPPING VIDEO: Trump Already Knows! He Alw...,It appears that the time for the BOOM to drop ...,1
4,2021-06-25,Biden Caught Openly Mocking Proud Gun Owners!,"During Joe Biden’s speech, he tried to hit at ...",1
...,...,...,...,...
2547,2021-04-08,"Alex Jones Saw Some Kids In Trouble, What He D...","In a now-viral video, conservative radio host ...",1
2548,2021-05-19,FRAUD Fauci Just Made An Absolutely SICKENING ...,Since Dr. Fauci has stepped onto the world sta...,1
2549,2021-06-24,"Biden Tweets, Then Mysteriously Deletes Haunti...",We all know that Joe Biden is not actually wri...,1
2550,2021-06-23,Trey Gowdy Has A Brutal Message For Democrats!,"In an interview with Fox News, Former Republic...",1


##Dcgazette

In [ ]:
import scrapy

class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://dcgazette.com/'
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        getLinkInDiv = response.css('h3.content-list-title')
        allLinks = getLinkInDiv.css('a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)
        next_page = response.css('a.next::attr(href)')[-1].get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']  
        available = False 
        date = response.css('.entry-meta span::text').get()  
        for year in yearsList:  
            if year in date:  
                available = True  
                break
        if available:
            title = response.css('.entry-title::text').get()
            paragraph = response.css('.entry-content p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 4:
                    break
                if count == 0:
                    count += 1
                    continue
                sentence.replace('\xa0', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 1
            }

In [ ]:
dcgaze = pd.read_json('https://raw.githubusercontent.com/nguyenthithaohien/data/main/data5.json')
Filter(dcgaze)
dcgaze

,date,title,text,is_fake
0,2020-08-21,Devout Catholic Joe Biden Tells America: “No ...,"Biden is a devout Catholic, or so :Biden’s ide...",1
1,2020-08-19,Trump Pardons Susan B. Anthony on 100th Anniv...,to mark the 100th anniversary of the to the ...,1
2,2020-08-19,ANOTHER Democrat Announces Support For Donald...,Now Leo has had a change of heart. He is suppo...,1
3,2020-08-19,Dumpster Fire: On Night 2 of the DNC Joe Bide...,Joe then introduced himself as ‘Joe Biden’s h...,1
4,2020-08-20,OMG! DNC Faked Convention Crowd! – Used Doubl...,But then somebody noticed the crowd was faked!...,1
...,...,...,...,...
1479,2019-04-01,Twitter Won’t Allow,the movie is based on the real story of Abby...,1
1480,2019-04-02,Rep. Ilhan Omar Under Investigation For Using...,"According to , authorities have recently compl...",1
1481,2019-04-01,Ginsburg Sighting! Justice Ruth Bader Ginsbur...,: It appears she has some assistance.,1
1482,2019-04-02,Jared Kushner: In Florida After Passing Law T...,on Monday to discuss the recent White House s...,1


##Worldtruth

In [ ]:
import scrapy

class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://worldtruth.tv/2018/'
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        getLinkInDiv = response.css('h3.entry-title')
        allLinks = getLinkInDiv.css('a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)
        next_page = response.css('.page-nav a::attr(href)')[-1].get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        title = response.css('.entry-title::text').get()
        paragraph = response.css('.td-post-content p::text').getall()
        text = ''
        count = 0
        for sentence in paragraph:
            if count == 4:
                break
            if count == 0:
                count += 1
                continue
            sentence.replace('\xa0', '')
            text = text + sentence
            count += 1
        yield {
            'date': 2018,
            'title': title,
            'text': text,
            'is_fake': 1
        }

In [ ]:
world2018 = pd.read_json('https://raw.githubusercontent.com/nguyenthithaohien/data/main/data7.json')
world2018.drop(world2018[world2018['text'] == " \n "].index, inplace=True)
Filter(world2018)
world2018

,date,title,text,is_fake
0,2018,"Cancer Linked Glyphosate Found In Cheerios, Do...",The site states that toddlers age 9 months and...,1
1,2018,"She Thought They Were Twins, But The Doctors S...","This was the case of Alexa and Antonin Kinova,...",1
2,2018,A Diet Guru Explains Why You Should Eat Dinner...,Another study shows that restricting meals to ...,1
3,2018,"Saltwater Powered Car Runs 1,000 km On A Full ...","Today, we are going to talk about a new soluti...",1
4,2018,Student Uses Coupon Clipping Skills To Buy $10...,", it turns out.A 16-year-old girl actually lea...",1
...,...,...,...,...
4593,2018,NORWAY Police Haven’t Killed In A Decade – Wha...,"As an adult, I can’t help but feel like my hom...",1
4594,2018,Anonymous Hackers Take 9 Rothschild Central Ba...,".” This massive push, according to the video, ...",1
4595,2018,Tennessee Titans Player Has One Word For Fans ...,The Titans the national anthem altogether befo...,1
4596,2018,Scientists Have Solved An Ancient Peruvian Mys...,The mystery centers around a series of careful...,1


In [ ]:
world2019 = pd.read_json('https://raw.githubusercontent.com/nguyenthithaohien/data/main/data8.json')
world2019.drop(world2019[world2019['text'] == " \n "].index, inplace=True)
Filter(world2019)
world2019

,date,title,text,is_fake
0,2019,JFK Files: Documents Show Democrat President L...,News outlets from around the globe are furious...,1
1,2019,Cancer Linked Monsanto Chemical Discovered In ...,this is why it will likelynever cure cancer b...,1
2,2019,Pharmaceutical Giants Caught Supplying Cartels...,The companies are accused of providing the Mex...,1
3,2019,Wall Street Journal Investigation Finds Amazon...,.The bombshell investigation reveals that “Lat...,1
4,2019,Japanese Workers Amazed The World By Fixing Gi...,"turned into an extremely massive one, and nob...",1
...,...,...,...,...
4407,2019,United Nations Exposes Chemtrails 100% Proof W...,By now everyone has witnessed streaks of white...,1
4408,2019,Climate Youth Puppet Greta Thunberg Is Control...,1. Polar bear populations are thriving\n2. Art...,1
4409,2019,The Top 10 Breakfast Cereals Most Likely To Co...,But the media has not yet reported on the ever...,1
4410,2019,"Did You Know The Democrats Ran The KKK, Starte...","American principles and values, but thanks to ...",1


In [ ]:
world2020 = pd.read_json('https://raw.githubusercontent.com/nguyenthithaohien/data/main/data10.json')
world2020.drop(world2020[world2020['text'] == " \n "].index, inplace=True)
Filter(world2020)
world2020

,date,title,text,is_fake
0,2020,Why 50% of The U.S. Population Will Not Surviv...,This doesn’t even consider the additional risk...,1
1,2020,Italian Doctor Shocked The World: Cancer Is A ...,The therapy isn’t harmful at all and let’s fac...,1
2,2020,This Pretty Girl Was Seeking A Rich Husband – ...,I’m going to be honest of what I’m going to sa...,1
3,2020,The UN Plans To Implement Universal Biometric ...,"For example, Goal 16.9 sets …“By 2030, provide...",1
4,2020,Here’s 100 Confirmed Conspiracies From The Las...,"” so stuff like Paperclip, MKUltra, Mockingbir...",1
...,...,...,...,...
2832,2020,Best Flat Earth Video 1000% Proof The Earth Is...,This is a very good question. For some time af...,1
2833,2020,The Great Reset,"In his book, , World Economic Forum (WEF) foun...",1
2834,2020,Experts Warn mRNA Vaccines Could Cause Irrever...,Chief among his concerns is the fact that the ...,1
2835,2020,Massive Underground Tunnels of Stone Age Stret...,Others suggest the linked tunnels were used as...,1


##Opindia

In [ ]:
import scrapy

class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://www.opindia.com/category/politics/',
            'https://www.opindia.com/category/opinions/',
            'https://www.opindia.com/category/fact-check/',
            'https://www.opindia.com/category/media/',
            'https://www.opindia.com/category/variety/',
            'https://www.opindia.com/category/explainer/',
            'https://www.opindia.com/category/virtual-world/',
            'https://www.opindia.com/category/entertainment/',
            'https://www.opindia.com/category/political-history-of-india/',
            'https://www.opindia.com/category/government-and-policy/',
            'https://www.opindia.com/category/economy-and-finance/',
            'https://www.opindia.com/category/sports/',
            'https://www.opindia.com/category/international/',
            'https://www.opindia.com/category/crime/',
            'https://www.opindia.com/category/law/'
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        getLinkInDiv = response.css('h3.td-module-title')
        allLinks = getLinkInDiv.css('a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)
        next_page = response.css('.page-nav a::attr(href)')[-1].get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']  
        available = False  
        date = response.css('div.tdb-block-inner time::text').get()  
        for year in yearsList:  
            if year in date:  
                available = True  
                break
        if available:
            title = response.css('.tdb-title-text::text').get()
            paragraph = response.css('.tdb-block-inner p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 4:
                    break
                if count == 0:
                    count += 1
                    continue
                sentence.replace('\xa0', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 1
            }

In [ ]:
opindia = pd.read_json('https://raw.githubusercontent.com/nguyenthithaohien/data/main/data2.json')
Filter(opindia)
opindia

,date,title,text,is_fake
0,2021-07-01,8 European nations accept Covishield in their ...,Seven EU nations and Switzerland have accepted...,1
1,2021-07-01,Dehradun: Bajrang Dal stops Welham school’s Ha...,"On June 26, a tender notice was published in t...",1
2,2021-06-30,World’s second-largest kids’ apparel manufactu...,"On June 29, Kitex Group, the largest private-s...",1
3,2021-06-30,74% Indian Muslims prefer Sharia over Indian l...,"On June 29, Pew Research Centre published find...",1
4,2021-06-30,Befitting reply? “Tapsee Pannu begs to produce...,In a not-so-pleasant morning for Bollywood ent...,1
...,...,...,...,...
22452,2018-10-02,Bollywood actor and serial abuser was invited ...,for sending obscene pics to women over a mess...,1
22453,2021-03-23,While trying to mock “Sanghis” on Bhagat Singh...,23rd March is observed as Martyr’s Day in Indi...,1
22454,2020-05-15,"Disgrace, Duality and Dirt behind the Kashmiri...",Events followed by that night had nothing to b...,1
22455,2018-09-14,"Thanks morons for the FIR, now hope AIB is dra...",and calls it “” (disgrace). It invites repres...,1


##Blingnews

In [ ]:
import scrapy

class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'http://www.blingnews.com/'
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        getLinkInDiv = response.css('h3.content-list-title')
        allLinks = getLinkInDiv.css('a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)
        next_page = response.css('a.next::attr(href)')[-1].get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']  
        available = False  
        date = response.css('span.entry-meta-date::text').get()  
        for year in yearsList: 
            if year in date: 
                available = True  
                break
        if available:
            title = response.css('.entry-title::text').get()
            paragraph = response.css('.entry-content p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 4:
                    break
                if count == 0:
                    count += 1
                    continue
                sentence.replace('\xa0', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 1
            }

In [ ]:
bling = pd.read_json('https://raw.githubusercontent.com/nguyenthithaohien/data/main/data15.json')
Filter(bling)
bling

,date,title,text,is_fake
0,2019-09-25,PHOTOS: Ivanka Trump Wears Ralph Lauren to Unv...,“Today we dedicated the new US Embassy in Jeru...,1
1,2019-09-25,Fashion Notes: Melania Trump Returns Sun-Kisse...,at Bergdorf Goodman.The floral Christian Loub...,1
2,2019-09-25,Fashion Notes: Melania Trump Returns to Manhat...,"Most notably, Mrs. Trump wore a pair of patent...",1
3,2019-09-29,8 Things You Should Let Go to Find Your Way Ba...,"in the many roles we play, in the many things...",1
4,2019-09-25,Fashion Notes: Melania Trump Gets Western in D...,The Slovenian-born former fashion model button...,1
...,...,...,...,...
1694,2018-01-02,Huma Abedin Forwarded State Dept Passwords to ...,Via Luke Rosiak of:Huma Abedin forwarded sensi...,1
1695,2018-01-01,Fashion Notes: Melania Trump Is Glitter Queen ...,"at Neiman Marcus.In head-to-toe sequins, Mela...",1
1696,2018-01-01,Fashion Notes: First Lady Melania Trump’s 10 M...,"Dolce & Gabbana, Chanel, Dior, Hervé Pierre, D...",1
1697,2018-01-01,WATCH – Crybaby Don Lemon Freaks Out Over Word...,"When they cannot flee to their safe space, the...",1


##RightWingTribune

In [ ]:
import scrapy

class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://rightwingtribune.com/'
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        getLinkInDiv = response.css('h3.content-list-title')
        allLinks = getLinkInDiv.css('a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)
        next_page = response.css('a.next::attr(href)')[-1].get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']  
        available = False
        date = response.css('span.entry-meta-date::text').get()  
        for year in yearsList:  
            if year in date:  
                available = True  
                break
        if available:
            title = response.css('.entry-title::text').get()
            title = title.replace('\n', '')
            title = title.replace('\t', '')
            title = title.replace('\r', '')
            paragraph = response.css('.entry-content p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 4:
                    break
                if count == 0:
                    count += 1
                    continue
                sentence.replace('\xa0', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 1
            }

In [ ]:
rightwing = pd.read_json('https://raw.githubusercontent.com/nguyenthithaohien/data/main/data16.json')
Filter(rightwing)
rightwing

,date,title,text,is_fake
0,2020-12-28,Democrat James Carville… “80% Of Democrats Are...,Former Bill Clinton campaign manager and Democ...,1
1,2020-12-26,BREAKING: FBI Moves On Dem Senator Dianne Fein...,That’s called insider information folks.Feinst...,1
2,2020-10-04,CLASSIC!!! Kayleigh McEnany Brilliantly Turns ...,"Like her predecessor, White House Press Secret...",1
3,2020-12-16,Seattle Radio Host Who Claims Protests Are Not...,Take for instance some of the MSM and Democrat...,1
4,2020-10-03,"Breaking Report: Schiff Is In Panic Mode, Care...","Schiff has finally met his match, someone who ...",1
...,...,...,...,...
6785,2020-10-01,"Prominent Dem Arrested, Charged With 6 Felony ...","Democrats… Let’s face it, the amount of corrup...",1
6786,2020-10-02,We Just Uncovered The Past Of The Host Of The ...,“The Central Intelligence Agency owns everyone...,1
6787,2020-10-02,BREAKING: New York Mayor Indicted And Charged ...,"Rochester, N.Y., Mayor Lovely Warren, was indi...",1
6788,2020-10-01,Watch As Michelle Obama Degrades The Office Of...,?Trotskyism is the theory of as advocated by ...,1


##EndOfTheAmericanDream

In [ ]:
import scrapy

class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'http://endoftheamericandream.com/'
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        getLinkInDiv = response.css('h2.entry-title')
        allLinks = getLinkInDiv.css('a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)
        next_page = response.css('a.next::attr(href)')[-1].get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']  
        available = False  
        date = response.css('span.posted-on time::text').get() 
        for year in yearsList: 
            if year in date: 
                available = True  
                break
        if available:
            title = response.css('.entry-title::text').get()
            title = title.replace('\n', '')
            title = title.replace('\t', '')
            title = title.replace('\r', '')
            paragraph = response.css('.entry-content p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 4:
                    break
                if count == 0:
                    count += 1
                    continue
                sentence.replace('\xa0', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 1
            }

In [ ]:
end = pd.read_json('https://raw.githubusercontent.com/nguyenthithaohien/data/main/data17.json')
Filter(end)
end

,date,title,text,is_fake
0,2021-06-29,It Is So Dry In California That The Drinking W...,The only thing that is going to fix this on a ...,1
1,2021-07-04,Giant Swarms Of Grasshoppers That Can Be Seen ...,…The National Weather Service (NWS) Glasgow sa...,1
2,2021-06-24,The Worst Northwest Heatwave In History Is Abo...,has already started in some parts of the nati...,1
3,2021-06-28,14 Astonishing Facts About The Blistering Heat...,"At this moment, the Northwest is being slammed...",1
4,2021-06-20,Major U.S. Cities Race To “Re-Fund The Police”...,that the number of homicides in the U.S. is u...,1
...,...,...,...,...
667,2018-01-03,"There Have Been More Than 30,000 Documented Te...",", more than 30,000 documented terror attacks h...",1
668,2018-05-16,Thank You,"Several months ago, the numbers told us that t...",1
669,2018-05-01,Watch The Most Controversial Congressional Deb...,…I honestly do not know how Fulcher’s campaign...,1
670,2018-05-14,Vote Pro-Trump On May 15th – Election Day Is T...,. And please contact every single Trump suppor...,1


##21centurywire

In [ ]:
import scrapy


class NewsSpider(scrapy.Spider):
    name = "news"
    page_number = 2
    def start_requests(self):
        urls = [
        'https://21stcenturywire.com/category/international-news/',
        'https://21stcenturywire.com/category/europe/',
        'https://21stcenturywire.com/category/africa/',
        'https://21stcenturywire.com/category/middle-east/',
        'https://21stcenturywire.com/category/us-news/',
        'https://21stcenturywire.com/tag/eurasia/',
        'https://21stcenturywire.com/category/south-america/',
        'https://21stcenturywire.com/category/sci-tech/',
        'https://21stcenturywire.com/tag/covid-19/'
       ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        getLinkinDiv = response.css('h2.entry-title')
        allLinks = getLinkinDiv.css('a::attr(href)').getall()


        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = response.css('.pagination-next a::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        title = response.css('.entry-title::text').get()
        title = title.replace('\n', '')
        title = title.replace('\t', '')
        date = response.css('.time::text').get()
        paragraph = response.css('p::text').getall()


        text = ''

        count = 0

        for sentence in paragraph:
            if count == 0:
                count += 1
                continue
            if count == 4:
                break
            sentence = sentence.replace('\xa0', '')
            sentence = sentence.replace('\n', '')
            sentence = sentence.replace('\t', '')
            text = text + sentence
            count += 1

        yield {
            'date' : date,
            'title' : title,
            'text' : text,
            'is_fake' : 1
        }

In [ ]:
stcenturywire = pd.read_json('https://raw.githubusercontent.com/hoainam2310/Machine-Learning/main/21stcenturywire1.json')
Filter(stcenturywire)
stcenturywire

,date,title,text,is_fake
0,2021-06-18,UKC News: The Government Won’t Let Go of Its P...,Co-host and with the end of week news round-...,1
1,2019-04-12,Battlefield Libya and the Fruits of US-NATO Re...,Forces under the control of Khalifa Haftar – a...,1
2,2019-04-12,Sudanese President al-Bashir Ousted in Militar...,Protesters were happy to hear that the 30-year...,1
3,2019-12-01,Empty Planet: The World’s Shrinking Population,In this fascinating and harrowing discussion o...,1
4,2019-07-22,Ethiopia: Ethnic Apartheid and the Globalist C...,What is evident as one scans the media landsca...,1
...,...,...,...,...
7764,2013-02-10,Glimmer of Hope: Washington Residents Force Se...,It’s safe to assume that after this week’s dev...,1
7765,2013-02-09,Obamacare: Pure Deception Against Working People,The article below is the most comprehensive an...,1
7766,2013-02-09,First Nuke In Currency Wars: Venezuela Launche...,"And that, ladies and gents of , is how you jus...",1
7767,2013-02-09,"TYRANNY VIA CELEBRITY: Bush, Celebrity ‘Hackin...",says… It’s shocking to witness how quickly an...,1


In [ ]:
#Chuyển kiểu dữ liệu về dạng string 
stcenturywire = stcenturywire.astype({"date":str})

In [ ]:
Delete_From_DataFrame(stcenturywire)
stcenturywire

,date,title,text,is_fake
0,2021-06-18,UKC News: The Government Won’t Let Go of Its P...,Co-host and with the end of week news round-...,1
1,2019-04-12,Battlefield Libya and the Fruits of US-NATO Re...,Forces under the control of Khalifa Haftar – a...,1
2,2019-04-12,Sudanese President al-Bashir Ousted in Militar...,Protesters were happy to hear that the 30-year...,1
3,2019-12-01,Empty Planet: The World’s Shrinking Population,In this fascinating and harrowing discussion o...,1
4,2019-07-22,Ethiopia: Ethnic Apartheid and the Globalist C...,What is evident as one scans the media landsca...,1
...,...,...,...,...
2654,2020-04-18,China’s New ‘Coronavirus’ Surveillance Grid: 2...,We warned about Soviet,1
2655,2020-04-13,"COVID-19 Panic Merchants: Lies, Conspiracies a...","This video was recorded by on April 2, 2020. ...",1
2656,2020-04-03,Dr. Bruce Lipton: ‘The Truth About This COVID ...,We warned about Soviet . Its ‘public health ex...,1
2657,2020-03-15,Episode #318 – ‘Outbreak: The Reality TV Show’...,Episode #318 of resumes on with host,1


##Infostormer 

In [ ]:
import scrapy


class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://infostormer.com/category/world/',
            'https://infostormer.com/category/featured/',
            'https://infostormer.com/category/us/',
            'https://infostormer.com/category/culture/',
            'https://infostormer.com/category/jewish-problem/'
        ]

        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        getLinkInDiv = response.css('div.td_module_11')
        allLinks = getLinkInDiv.css('.entry-title a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = response.css('.page-nav a::attr(href)')[-1].get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse, dont_filter=True)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']
        available = False
        date = response.css('.entry-date::text').get()
        for year in yearsList:
            if year in date:
                available = True
                break
        if available:
            title = response.css('.entry-title::text').get()
            paragraph = response.css('.td-post-content p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 3:
                    break
                sentence = sentence.replace('\xa0', '')
                sentence = sentence.replace('\n', '')
                sentence = sentence.replace('\r', '')
                sentence = sentence.replace('\t', '')
                sentence = sentence.replace('\\', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 1
            }


In [ ]:
infos = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/infostormer.json')
Filter(infos)
infos

,date,title,text,is_fake
0,2021-06-24,NFL Player Praised for Doing Gay Anal Sex,A defensive lineman for the Oakland Raiders na...,1
1,2021-06-26,Pulse Nightclub is Now a National Memorial,"Back in 2016, a gay nightclub called Pulse in ...",1
2,2021-06-24,McDonald’s Offering Free COVID Shots,McDonald’s locations throughout California are...,1
3,2021-06-27,Climate Change Turns Al Gore Into an Orange Prune,Al Gore was recently on to complain about Don...,1
4,2021-06-26,Stanford Scientist Shills “Superhero” Vaxx,A Stanford scientist is shilling some type of ...,1
...,...,...,...,...
5708,2018-01-05,Sargon of Akkad Destroyed By Richard Spencer i...,One of the big happenings on the tubes last ni...,1
5709,2018-01-04,Steve Bannon’s Political Brand is Imploding,Steve Bannon’s political brand is imploding. O...,1
5710,2018-01-04,Is Global Warming to Blame for America Freezin...,For years I was told that the planet was warmi...,1
5711,2018-01-01,Stefan Molyneux Cries as He Promotes ZOG Revol...,"As we have been discussing, the Alt-Lite aka M...",1


##InvestmentWatch

In [ ]:
import scrapy
offset = 1
valid = True


class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://www.investmentwatchblog.com/'
        ]

        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        global offset
        allLinks = response.css('h2.entry-title a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)

        if valid:
            next_page = 'https://www.investmentwatchblog.com/page/'
            offset += 1
            next_page = next_page + str(offset) + '/'
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        global valid
        yearsList = ['2021', '2020', '2019', '2018']
        available = False
        date = response.css('.entry-date::text').get()
        if date is not None:
            for year in yearsList:
                if year in date:
                    available = True
                    break
        if available:
            title = response.css('.entry-title::text').get()
            paragraph = response.css('.entry-content p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 3:
                    break
                sentence = sentence.replace('\xa0', '')
                sentence = sentence.replace('\n', '')
                sentence = sentence.replace('\r', '')
                sentence = sentence.replace('\t', '')
                sentence = sentence.replace('\'', ' ')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 1
            }
        elif date is not None:
            valid = False

File's link: https://drive.google.com/file/d/1zaFO2kkaqMlTt4nyIG_O2EsHeajggSpB/view?usp=sharing

In [ ]:
invest = pd.read_json('drive/MyDrive/invest.json')
Filter(invest)
invest

,date,title,text,is_fake
0,2021-07-07,SURVEY: MALE FRIENDSHIP IN CRISIS… 1 in 5 have...,"After a prolonged period of social isolation, ...",1
1,2021-07-07,PRESSURE GROWS ON ANTI-VAXXERS… MANDATES AT WORK?,A growing number of countries and territories ...,1
2,2021-07-07,Los Angeles Is Squandering $1.2 Billion While ...,Federal Judge David O. Carter says Los Angeles...,1
3,2021-07-07,Jordan Roy Byrne – Cup & Handle In Gold Is Pur...,", Released on 7/6/21We are joined on Metal Mon...",1
4,2021-07-07,The Backward K and Coming Earnings Surprises,by The broad economic shutdowns and psychologi...,1
...,...,...,...,...
78385,2018-03-29,"Consumer Maxed Out, Goldilocks Fading, 30% Sto...","from .Steen Jakobsen, the often-bearish chief...",1
78386,2018-04-01,"CHILE, WORLD’S FOURTH LARGEST SILVER PRODUCER:...",BYSilver mine supply from the world’s fourth-l...,1
78387,2018-04-19,Warren Buffet just won his ten-year bet about ...,"“Over the years, I’ve often been asked for inv...",1
78388,2018-04-02,Financial CEO Gives Three Reasons Why Gold Wil...,"This week,rounds up the latest stories involvi...",1


##Madworld

In [ ]:
import scrapy
valid = True


class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        global valid
        urls = [
            'https://madworldnews.com/politics/',
            'https://madworldnews.com/social-issues/',
            'https://madworldnews.com/entertainment/',
            'https://madworldnews.com/health-science/',
            'https://madworldnews.com/animals/'
        ]

        for url in urls:
            valid = True
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        allLinks = response.css('.entry-title a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)

        if valid:
            next_page = response.css('a.next::attr(href)').get()
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        global valid
        yearsList = ['2021', '2020', '2019', '2018']
        available = False
        date = response.css('.entry-meta-date a::text').get()
        if date is not None:
            for year in yearsList:
                if year in date:
                    available = True
                    break
        if available:
            title = response.css('.entry-title::text').get()
            paragraph = response.css('.entry-content p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 3:
                    break
                sentence = sentence.replace('\xa0', '')
                sentence = sentence.replace('\n', '')
                sentence = sentence.replace('\r', '')
                sentence = sentence.replace('\t', '')
                sentence = sentence.replace('\'', ' ')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 1
            }
        elif date is not None:
            valid = False

In [ ]:
mad = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/madworld.json')
Filter(mad)
mad

,date,title,text,is_fake
0,2020-12-18,‘Brilliant Move’ By Founding Fathers Could Bec...,"According to recent polls, many Americans beli...",1
1,2020-12-18,Bombshell: Defense Secretary Orders Halt To Co...,According to officials across the Defense Depa...,1
2,2020-12-16,General Calls To Invoke ‘Insurrection Act’; Tr...,Retired Air Force Lt. Gen. Thomas McInerney is...,1
3,2020-12-16,"Democrats Freak Out, Theory For Trump’s ‘Paths...",Rep. Maxine Waters and Hillary Clinton are two...,1
4,2020-12-15,MI Election Officials Slam Dominion Forensic R...,Michigan election officials are slamming the f...,1
...,...,...,...,...
82,2018-01-17,Arizona Teen Makes Unnerving Find Dangling Fro...,A 15-year-old girl was walking behind a local ...,1
83,2018-03-13,Students ‘Traumatized’ After Seeing What Idaho...,Students from a school in Idaho have been left...,1
84,2018-03-07,"Cops Find Dog Abandoned On Road, Left Sickened...","In Atlanta, Georgia, the police received a cal...",1
85,2018-07-11,VIDEO: Nursing Student Goes ‘Swimming With Sha...,Katarina Zarutskie is a Miami nursing student ...,1


##3percent

In [ ]:
import scrapy
valid = True


class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        global valid
        urls = [
            'https://threepercenternation.com/'
        ]

        for url in urls:
            valid = True
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        allLinks = response.css('a.vw-post-box__link::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)

        if valid:
            next_page = response.css('a.next::attr(href)').get()
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        global valid
        yearsList = ['2021', '2020', '2019', '2018']
        available = False
        date = response.css('.vw-post-date time::text').get()
        if date is not None:
            for year in yearsList:
                if year in date:
                    available = True
                    break
        if available:
            title = response.css('.vw-post-title::text').get()
            paragraph = response.css('.vw-post-content p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 3:
                    break
                sentence = sentence.replace('\xa0', '')
                sentence = sentence.replace('\n', '')
                sentence = sentence.replace('\r', '')
                sentence = sentence.replace('\t', '')
                sentence = sentence.replace('\'', ' ')
                if sentence != "":
                    text = text + sentence
                    count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 1
            }
        elif date is not None:
            valid = False

In [ ]:
three = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/three.json')
Filter(three)
three

,date,title,text,is_fake
0,2021-07-02,DEMOCRAT Congressional Candidate INDICTED!,We all know that the Democrats are nothing but...,1
1,2021-07-01,FBI Agent ARRESTED In Sickening Abuse Of A Child!,We can still remember how great and well-respe...,1
2,2021-07-04,WHY IS BIDEN DEPLOYING RESPONSE TEAMS TO COMMU...,The Biden administration is not happy that the...,1
3,2021-07-04,AUDIT UPDATE: AZ Senate Planning On Issuing SU...,The Arizona audit is not a recount but an actu...,1
4,2021-07-04,Was Ashli Babbitt’s Shooter Accidentally REVEA...,"Oooo, things are heating up surrounding the ho...",1
...,...,...,...,...
3405,2018-01-03,SHOCKING: Chicago Woman Shot While Streaming o...,It is a well-known fact that Chicago has some ...,1
3406,2018-01-04,"Somalia Immigrant Rapes Woman, The Details Wil...",Rape is a horrible crime because the memory of...,1
3407,2018-01-03,"Mind Blowing: Scientists Defeat Extinction, Ar...",We live in a pretty incredible time. There was...,1
3408,2018-01-04,BOMBSHELL: Proof That Comey Gave Classified Me...,Chuck Grassley used simple math to prove that ...,1


#Tổng hợp trang tin giả

In [ ]:
fake_news_website = [zerohedge, prntly, thegateway, conservative, dailyheadline, dcgaze, world2018, world2019, world2020, opindia, bling, rightwing, end, stcenturywire, infos, invest, mad, three]
data_fake_news = activist.append(natural, ignore_index=True)
for item in fake_news_website:
  data_fake_news = data_fake_news.append(item, ignore_index=True)
data_fake_news

,date,title,text,is_fake
0,2021-05-28 00:00:00,US Mint Delays Silver Shipments Due To “Global...,Interest in silver is soaring (both for indust...,1
1,2021-06-28 00:00:00,"Research Paper Exposes Cybersecurity, Environm...",A 2018 survey revealed did NOT want to live i...,1
2,2021-05-29 00:00:00,DeSantis’s Anti-Riot Law Undermines Two Import...,When Florida Gov. Ron DeSantis spent the last ...,1
3,2021-05-28 00:00:00,Another Massive Cargo Ship Was Just Stuck In t...,To quote the great Los Angeles sportscaster Vi...,1
4,2021-06-25 00:00:00,BofA Crashes The “Transitory” Party: Sees Up T...,"At the start of May, when observing the avalan...",1
...,...,...,...,...
202477,2018-01-03 00:00:00,SHOCKING: Chicago Woman Shot While Streaming o...,It is a well-known fact that Chicago has some ...,1
202478,2018-01-04 00:00:00,"Somalia Immigrant Rapes Woman, The Details Wil...",Rape is a horrible crime because the memory of...,1
202479,2018-01-03 00:00:00,"Mind Blowing: Scientists Defeat Extinction, Ar...",We live in a pretty incredible time. There was...,1
202480,2018-01-04 00:00:00,BOMBSHELL: Proof That Comey Gave Classified Me...,Chuck Grassley used simple math to prove that ...,1


#Trang tin chính thống

##Economist

In [ ]:
import scrapy


class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://www.economist.com/leaders',
            'https://www.economist.com/briefing',
            'https://www.economist.com/united-states',
            'https://www.economist.com/asia',
            'https://www.economist.com/the-americas',
            'https://www.economist.com/china',
            'https://www.economist.com/middle-east-and-africa',
            'https://www.economist.com/europe',
            'https://www.economist.com/britain',
            'https://www.economist.com/international',
            'https://www.economist.com/business',
            'https://www.economist.com/finance-and-economics',
            'https://www.economist.com/science-and-technology',
            'https://www.economist.com/books-and-arts',
            'https://www.economist.com/graphic-detail',
            'https://www.economist.com/obituary'
        ]

        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        getLinkInDiv = response.css('div.teaser__text')
        allLinks = getLinkInDiv.css('a::attr(href)').getall()

        for each_link in allLinks:
            each_link = 'https://www.economist.com' + each_link
            yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = response.css('.ds-pagination__nav--next a::attr(href)')[0].get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']
        available = False
        date = response.css('.article__dateline-datetime::text').get()
        for year in yearsList:
            if year in date:
                available = True
                break
        if available:
            title = response.css('.article__headline::text').get()
            paragraph = response.css('.article__body-text::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 3:
                    break
                sentence = sentence.replace('\xa0', '')
                sentence = sentence.replace('\n', '')
                sentence = sentence.replace('\r', '')
                sentence = sentence.replace('\t', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 0
            }

In [ ]:
eco = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/economist.json')
Filter(eco)
eco

,date,title,text,is_fake
0,2021-06-24,We’re hiring: a news assistant in Tokyo,"is seeking a . This is an exciting, multiface...",0
1,2021-06-24,An important census product may soon use synth...,"(), which is sent to around 1% of America’s p...",0
2,2021-06-24,Myanmar’s civil war is becoming bloodier and m...,"bicycle, wearing a -shirt emblazoned with a c...",0
3,2021-06-24,America’s Supreme Court is less one-sided than...,America’s Supreme Court seemed destined for a...,0
4,2021-06-12,The anti-graft unit of China’s Communist Party...,"June 1st Shi Zhaoqing, a local boss in China’...",0
...,...,...,...,...
12587,2020-08-20,"For your summer getaway, try an imaginary city","a summer getaway? Try the city of Isidora, “w...",0
12588,2019-01-17,Indonesia’s economic growth is being held back...,"in Singapore on January 15th, Indonesian offi...",0
12589,2018-09-20,The leaders of the two Koreas put on another g...,"THE setting keeps changing; the pictures, not ...",0
12590,2018-04-14,Kinder Morgan’s attempt to build a pipeline re...,"ALMOST all Canada’s oil and gas is landlocked,...",0


##APNews

In [ ]:
import scrapy


class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://apnews.com/hub/coronavirus-pandemic',
            'https://apnews.com/hub/politics',
            'https://apnews.com/hub/sports',
            'https://apnews.com/hub/entertainment',
            'https://apnews.com/hub/lifestyle',
            'https://apnews.com/hub/oddities',
            'https://apnews.com/hub/travel',
            'https://apnews.com/hub/technology',
            'https://apnews.com/hub/ap-fact-check',
            'https://apnews.com/hub/business',
            'https://apnews.com/hub/us-news',
            'https://apnews.com/hub/health',
            'https://apnews.com/hub/science',
            'https://apnews.com/hub/world-news',
            'https://apnews.com/hub/religion'
        ]

        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        getLinkInDiv = response.css('div.CardHeadline')
        allLinks = getLinkInDiv.css('a::attr(href)').getall()

        for each_link in allLinks:
            each_link = 'https://apnews.com' + each_link
            yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = response.css('.ds-pagination__nav--next a::attr(href)')[0].get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']
        available = False
        date = response.css('.Timestamp::attr(title)').get()
        temp = date.split(' ', 1)
        date = temp[0]
        for year in yearsList:
            if year in date:
                available = True
                break
        if available:
            title = response.css('.CardHeadline h1::text').get()
            paragraph = response.css('.Article p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 3:
                    break
                sentence = sentence.replace('\xa0', '')
                sentence = sentence.replace('\n', '')
                sentence = sentence.replace('\r', '')
                sentence = sentence.replace('\t', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 0
            }

In [ ]:
apnews = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/apnews.json')
Filter(apnews)
apnews

,date,title,text,is_fake
0,2021-07-02,Dogs on display: Museum fetes 200 years of car...,"COLUMBUS, Ohio (AP) — In a 1970 Beetle Bailey ...",0
1,2021-07-03,"Ransomware hits hundreds of US companies, secu...",WASHINGTON (AP) — A ransomware attack paralyze...,0
2,2021-07-02,Paris airport workers block terminal to protes...,PARIS (AP) — Paris airport workers protesting ...,0
3,2021-07-02,‘Normal kid’ Grealish an England fan favorite ...,"BURTON-ON-TRENT, England (AP) — Fresh from an ...",0
4,2021-05-19,"UN urges more vaccines for Africa, with only 2...",UNITED NATIONS (AP) — The U.N. Security Counci...,0
...,...,...,...,...
721,2021-07-01,VIRUS DIARY: The unfinished business of a fune...,"NASHVILLE, Tenn. (AP) — It dawned on me recent...",0
722,2021-07-01,Whither #MeToo? Chilling effect of Cosby rever...,"When Indira Henard, director of the DC Rape Cr...",0
723,2021-07-02,AP Week in Pictures: North America,"JUNE 25 - JULY 1, 2021This photo gallery highl...",0
724,2021-07-03,Restaurateurs contend with new challenges for ...,"NEW KENSINGTON, Pa. (AP) — Restaurants were hi...",0


##Npr (**load_more style**)

In [ ]:
import scrapy
offset, times = 0, 0
valid = True
res = ''
dictionary = {
    'national/': [1003, 25],
    'politics/': [1014, 25],
    'health/': [1128, 23],
    'technology/': [1019, 24],
    'world/': [1004, 22],
    'business/': [1006, 25],
    'science/': [1007, 24]
}


class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        global times, res, offset, valid
        urls = [
            'https://www.npr.org/sections/national/',
            'https://www.npr.org/sections/politics/',
            'https://npr.org/sections/health/'
            'https://npr.org/sections/technology/'
            'https://www.npr.org/sections/world/'
            'https://www.npr.org/sections/business/'
            'https://www.npr.org/sections/science/',
        ]

        for url in urls:
            for key in dictionary.keys():
                if url.endswith(key):
                    res = key
                    break
            times = 0
            offset = 0
            valid = True
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        global offset, times
        getLinkInDiv = response.css('h2.title')
        allLinks = getLinkInDiv.css('a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)

        if valid:
            next_page = 'https://www.npr.org/get/'
            temp = dictionary[res][0]
            next_page = next_page + str(temp)
            next_page = next_page + '/render/partial/next?start='
            if times == 0:
                offset += dictionary[res][1]
                times += 1
            else:
                offset += 24
            next_page = next_page + str(offset)
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        global valid
        yearsList = ['2021', '2020', '2019', '2018']
        available = False
        date = response.css('.date::text').get()
        for year in yearsList:
            if year in date:
                available = True
                break
        if available:
            title = response.css('.storytitle h1::text').get()
            paragraph = response.css('.storylocation p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 0 or count == 1 or count == 2:
                    count += 1
                    continue
                if count == 6:
                    break
                sentence = sentence.replace('\xa0', '')
                sentence = sentence.replace('\n', '')
                sentence = sentence.replace('\r', '')
                sentence = sentence.replace('\t', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 0
            }
        else:
            valid = False

In [ ]:
npr_national = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/npr_national.json')
npr_business = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/npr_business.json')
npr_science = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/npr_science.json')
npr_health = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/npr_health.json')
npr_tech = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/npr_tech.json')
npr_world = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/npr_world.json')
npr_politic = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/npr_politics.json')

In [ ]:
npr_list = [npr_science, npr_health, npr_tech, npr_world, npr_politic]
npr = npr_national.append(npr_business, ignore_index=True)
for i in npr_list:
  npr = npr.append(i, ignore_index=True)
Filter(npr)
npr

,date,title,text,is_fake
0,"Updated July 2, 2021",Judge Approves Company's Withdrawal From Co-Ma...,A #FreeBritney activist protests against keepi...,0
1,"Updated July 2, 2021",Boy Scouts Of America Reaches Historic Settlem...,The Boy Scouts of America has reached a settle...,0
2,"Updated July 1, 2021","Trump's Family Business, CFO Weisselberg Are C...","Allen Weisselberg, the Trump Organization's lo...",0
3,"Updated July 1, 2021",The Justice Department Is Pausing Federal Exec...,Attorney General Merrick Garland ordered a pau...,0
4,"July 1, 2021",Judge Orders Release Of Wisconsin Woman In Sle...,"Anissa Weier, pictured in 2017, one of two Wis...",0
...,...,...,...,...
30471,"March 12, 2020",Sanders Offers Biden A Path To Win Over His Mo...,"After big Democratic primary losses, Bernie Sa...",0
30472,"March 12, 2020",Pelosi Vows To Bring Coronavirus Bill To House...,House Speaker Nancy Pelosi was unable to get a...,0
30473,"March 12, 2020","Trump Defends Travel Ban, Says Stock Market Wi...",President Trump and Irish Prime Minister Leo V...,0
30474,"March 13, 2020","Trump, Pelosi Agree On Coronavirus Relief Bill...","House Majority Leader Steny Hoyer, D-Md., is q...",0


##Brg

In [ ]:
import scrapy

class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://bgr.com/tech/',
            'https://bgr.com/entertainment/',
            'https://bgr.com/deals/',
            'https://bgr.com/business/',
            'https://bgr.com/science/',
            'https://bgr.com/lifestyle/'
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        getLinkInDiv = response.css('h3.font-bold')
        allLinks = getLinkInDiv.css('a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)
        next_page = response.css('a.next::attr(href)')[-1].get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']  
        available = False  
        date = response.css('.lg\:mr-0 div::text').get()  
        for year in yearsList: 
            if year in date: 
                available = True  
                break
        if available:
            title = response.css('.entry-title::text').get()
            paragraph = response.css('.entry-content p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 4:
                    break
                if count == 0:
                    count += 1
                    continue
                sentence.replace('\xa0', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 1
            }

In [ ]:
brg = pd.read_json('https://raw.githubusercontent.com/nguyenthithaohien/data/main/data3.json')
Filter(brg)
brg

,date,title,text,is_fake
0,2021-07-01 20:24:00,Waze’s latest update is perfect for dog and ca...,about the traffic ahead. You can use Waze as ...,0
1,2021-06-30 15:21:00,Apple Watch fall detection feature helped save...,"Since its debut, we’ve seen several stories in...",0
2,2021-06-30 16:35:00,Will Apple’s iPhone 13 release be delayed? Her...,. The consensus seems to be that one of the mo...,0
3,2021-06-30 17:12:00,WhatsApp ‘view once’ messages are rolling out ...,gave a hacker access to information from more...,0
4,2021-06-30 18:15:00,Phones with selfie cameras behind the screen c...,Phones with under-display cameras aren’t the s...,0
...,...,...,...,...
30630,2019-12-03 19:06:00,Baby Yoda could become Kid Yoda in future ‘Sta...,on Disney+ stands out. Not just because it’s ...,0
30631,2019-12-03 19:37:00,Netflix’s movie library is shrinking as compan...,"In light of that, Netflix hasn’t been afraid t...",0
30632,2019-12-04 12:50:00,Spider-Man might take his first step toward le...,films of its own — it will lose access to the...,0
30633,2019-12-04 09:53:00,Huge leak tells us how ‘Star Wars: The Rise of...,for — we’re in for six amazing cameos in Dis...,0


##Indiaexpress

In [ ]:
import scrapy

class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://indianexpress.com/section/world/',
            'https://indianexpress.com/section/india/',
            'https://indianexpress.com/section/cities/',
            'https://indianexpress.com/section/opinion/',
            'https://indianexpress.com/section/sports/',
            'https://indianexpress.com/section/entertainment/',
            'https://indianexpress.com/section/lifestyle/',
            'https://indianexpress.com/section/technology/',
            'https://indianexpress.com/section/explained/',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        getLinkInDiv = response.css('div.north-east-grid')
        allLinks = getLinkInDiv.css('a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)
        next_page = response.css('a.next::attr(href)')[-1].get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']  
        available = False  
        date = response.css('div.editor span::text').get()  
        for year in yearsList: 
            if year in date: 
                available = True  
                break
        if available:
            title = response.css('div.heading-part h1::text').get()
            paragraph = response.css('div#pcl-full-content p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 4:
                    break
                if count == 0:
                    count += 1
                    continue
                sentence.replace('\xa0', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 0
            }

In [ ]:
india = pd.read_json('https://raw.githubusercontent.com/nguyenthithaohien/data/main/data13.json')
Filter(india)
india

,date,title,text,is_fake
0,"July 4, 2021 8:36:13 am",Europe in vaccination race against COVID-19’s ...,vaccinations and outpace the spread of the mo...,0
1,"July 4, 2021 8:46:49 am","Palestinian killed in West Bank clash, retalia...",An Israeli military spokesperson said that sol...,0
2,"Updated: July 2, 2021 5:00:26 pm",UAE prohibits citizens from travelling to 14 c...,.The UAE’s Ministry of Foreign Affairs and In...,0
3,"July 2, 2021 8:20:51 pm",Boeing 737 cargo plane makes emergency landing...,“The pilots had reported engine trouble and we...,0
4,"July 3, 2021 7:50:03 am",British PM Johnson says AstraZeneca’s India Co...,vaccines should be left out of vaccine passpo...,0
...,...,...,...,...
28771,"Updated: September 9, 2018 9:10:15 pm",China detaining Muslims in vast numbers. The g...,"Abdusalam Muhemet, 41, said the police detaine...",0
28772,"September 9, 2018 10:57:57 am",Turkmenistan opens plant to ship power to Afgh...,The Saturday start of the plant near Mary in t...,0
28773,"Updated: September 9, 2018 11:17:35 am",Watergate memories spring to life with Donald ...,A president feels besieged by tormentors — Bob...,0
28774,"September 9, 2018 11:28:43 am",White House narrows search for anonymous op-ed...,Trump is still “obsessed” with finding the per...,0


##Slate

In [ ]:
import scrapy


class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://slate.com/news-and-politics/',
            'https://slate.com/culture/',
            'https://slate.com/technology/',
            'https://slate.com/culture/'
        ]
        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        allLinks = response.css('.topic-stories-list a::attr(href)').getall()

        for each_link in allLinks:
            each_link = response.urljoin(each_link)
            yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = response.css('.topic-stories-list__pagination a::attr(href)')[-1].get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse, dont_filter=True)

    def parse_href(self, response):
        var = response.css('.article__rubric::text').get()
        if var != '      The Slate Quiz':
            yearsList = ['2021', '2020', '2019', '2018']
            available = False
            date = response.css('.article__dateline::attr(content)').get()
            if date is not None:
                for year in yearsList:
                    if year in date:
                        available = True
                        break
            if available:
                title = response.css('h1.article__hed::text').get()
                title = title.replace('\n', '')
                title = title.replace('\t', '')
                paragraph = response.css('.article__content p::text').getall()
                text = ''
                count = 0
                for sentence in paragraph:
                    if count == 3:
                        break
                    sentence = sentence.replace('\xa0', '')
                    sentence = sentence.replace('\n', '')
                    sentence = sentence.replace('\t', '')
                    text = text + sentence
                    count += 1
                yield {
                    'date': date,
                    'title': title,
                    'text': text,
                    'is_fake': 0
                }

In [ ]:
slate = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/slate.json')
Filter(slate)
slate

,date,title,text,is_fake
0,2021-06-29T20:58:12+00:00,Every State Should Adopt Maine’s New Climate P...,"In a small-but-historic move, Maine just becam...",0
1,2021-07-01T09:45:02+00:00,Where’s My Lyme Vaccine?,Here’s a fun game I’ve played with fellow wood...,0
2,2021-06-30T16:19:01+00:00,How to Save the Case Against Facebook,The effort to crack down on the power of Faceb...,0
3,2021-06-29T18:32:31+00:00,Why the Pentagon Can’t Identify Flying Objects,"On Oct. 4, 1918, the U.S. Army Signal Corps su...",0
4,2021-06-30T13:00:00+00:00,Green Like AstroTurf—or Dollars,The good news for Mexico’s Green Party is that...,0
...,...,...,...,...
19708,2021-06-28T14:30:00+00:00,Texas Republicans Who Want to Lure Bitcoin Min...,China was at one point home to nearly . But no...,0
19709,2021-07-05T09:45:00+00:00,The Mystery of the Hypersonic Tic Tac,A long-awaited report on UFOs—what the governm...,0
19710,2021-07-02T19:48:26+00:00,What Happened When I Tried to Register as Dona...,The world of MAGA social networks is starting ...,0
19711,2021-06-29T15:08:20+00:00,“Bitcoin Beach” Is Scaling Up. Will Bitcoin Co...,Alongside the typical touristy surf-town sight...,0


##People

In [ ]:
import scrapy


class NewsSpider(scrapy.Spider):
    name = "news"
    page_number = 2
    def start_requests(self):
        urls = [
        'https://people.com/tag/news/?page=2',
        'https://people.com/entertainment/?page=2',
        'https://people.com/tag/coronavirus/?page=2',
        'https://people.com/royals/?page=2',
        'https://people.com/lifestyle/?page=2',
        'https://people.com/tag/shopping/?page=2'
       ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        getLinkinDiv = response.css('div.tout__contentHeadline')
        allLinks = getLinkinDiv.css('a::attr(href)').getall()


        for each_link in allLinks:
            each_link = 'https://people.com/' + each_link
            yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = response.css('a.category-page-list-related-nav-next-button::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        title = response.css('h1::text').get()
        title = title.replace('\n', '')
        title = title.replace('\t', '')
        date = response.css('.padding-12-left::text').get()
        paragraph = response.css('p::text').getall()


        text = ''

        count = 0

        for sentence in paragraph:
            if count == 0:
                count += 1
                continue
            if count == 4:
                break
            sentence = sentence.replace('\xa0', '')
            sentence = sentence.replace('\n', '')
            sentence = sentence.replace('\t', '')
            text = text + sentence
            count += 1

        yield {
            'date' : date,
            'title' : title,
            'text' : text,
            'is_fake' : 0
        }

In [ ]:
people = pd.read_json('https://raw.githubusercontent.com/hoainam2310/Machine-Learning/main/people1.json')
Filter(people)
people

,date,title,text,is_fake
0,"July 02, 2021 03:30 PM",Angelina Jolie and The Weeknd Step Out for a F...,and were spotted having a friendly night out...,0
1,"July 02, 2021 06:30 PM",Ohio Police Chief Leaves Job After He's Filmed...,The Ohio police chief who was filmed placing a...,0
2,"July 02, 2021 03:17 PM",Boyz II Men's Shawn Stockman on Raising a Chil...,Boyz II Men founding member is opening up abo...,0
3,"July 02, 2021 03:20 PM","Meet Wally Funk, the 82-Year-Old Woman Joining...",is heading to space! announced Thursday that...,0
4,"July 02, 2021 03:52 PM",Gordon Ramsay Faces Backlash Over His Attempt ...,is facing backlash over a recent episode of h...,0
...,...,...,...,...
38432,"July 02, 2021 05:32 PM",ASPCA Helps North Carolina Truck Driver Reunit...,Blue is back in his owner's arms! According to...,0
38433,"July 02, 2021 06:13 PM",Jamie Lynn Spears Says 'Stop with the Death Th...,is asking people to quit sending her and her ...,0
38434,"July 02, 2021 06:12 PM",Actress Jessie Cave Describes 'Uncomfortable E...,"films, said that her experience on set was di...",0
38435,"July 02, 2021 06:29 PM",Erika and Tom Girardi: Everything We Know Abou...,"star filed for divorce in November, the forme...",0


In [ ]:
Delete_From_DataFrame(people)
people

,date,title,text,is_fake
0,"July 02, 2021 03:30 PM",Angelina Jolie and The Weeknd Step Out for a F...,and were spotted having a friendly night out...,0
1,"July 02, 2021 06:30 PM",Ohio Police Chief Leaves Job After He's Filmed...,The Ohio police chief who was filmed placing a...,0
2,"July 02, 2021 03:17 PM",Boyz II Men's Shawn Stockman on Raising a Chil...,Boyz II Men founding member is opening up abo...,0
3,"July 02, 2021 03:20 PM","Meet Wally Funk, the 82-Year-Old Woman Joining...",is heading to space! announced Thursday that...,0
4,"July 02, 2021 03:52 PM",Gordon Ramsay Faces Backlash Over His Attempt ...,is facing backlash over a recent episode of h...,0
...,...,...,...,...
34555,"July 02, 2021 05:32 PM",ASPCA Helps North Carolina Truck Driver Reunit...,Blue is back in his owner's arms! According to...,0
34556,"July 02, 2021 06:13 PM",Jamie Lynn Spears Says 'Stop with the Death Th...,is asking people to quit sending her and her ...,0
34557,"July 02, 2021 06:12 PM",Actress Jessie Cave Describes 'Uncomfortable E...,"films, said that her experience on set was di...",0
34558,"July 02, 2021 06:29 PM",Erika and Tom Girardi: Everything We Know Abou...,"star filed for divorce in November, the forme...",0


##Hollywoodlife

In [ ]:
import scrapy


class NewsSpider(scrapy.Spider):
    name = "news"
    page_number = 2
    def start_requests(self):
        urls = [
            'https://hollywoodlife.com/topics/news/'
       ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        getLinkinDiv = response.css('div.article-feed__article-details')
        allLinks = getLinkinDiv.css('a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = 'https://hollywoodlife.com/topics/news/page/' + str(NewsSpider.page_number) + '/'
        if NewsSpider.page_number <= 10000:
            NewsSpider.page_number += 1
            yield response.follow(next_page, callback=self.parse)

    def parse_href(self, response):
        title = response.css('i::text').get()
        title = title.replace('\n', '')
        title = title.replace('\t', '')
        date = response.css('.article-header__eyebrow-item--date .article-header__eyebrow-text::text').get()
        paragraph = response.css('p::text').getall()

        text = ''

        count = 0

        for sentence in paragraph:
            if count == 0:
                count += 1
                continue
            if count == 4:
                break
            sentence.replace('\xa0', '')
            text = text + sentence
            count += 1

        yield {
            'date' : date,
            'title' : title,
            'text' : text,
            'is_fake' : 0
        }

In [ ]:
hollywoodlife = pd.read_json('https://raw.githubusercontent.com/hoainam2310/Machine-Learning/main/hollywoodlife1.json')
hollywoodlife

/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


,date,title,text,is_fake
0,2021-06-30 19:30:00,Bebe Rexha Proudly Rocks Lingerie To Promote B...,", 31, wants fans to know that she loves her bo...",0
1,2021-06-30 19:14:00,Jill Biden & Doug Emhoff Enjoy Beers Together ...,Cheers to baseball and COVID-19 vaccinations. ...,0
2,2021-07-01 00:23:00,Kevin Federline Denies Using His Kids As ‘Pawn...,‘s lawyer is making it clear he did not use hi...,0
3,2021-06-30 18:06:00,Prince Harry Admits It’s ‘A Juggle’ With 2 Kid...,Fellow British redheads and shared parenting ...,0
4,2021-06-30 19:12:00,Simone Biles Beats Boyfriend Jonathan Owens In...,"Not so fast! NFL player , 25, challenged girlf...",0
...,...,...,...,...
37625,2012-05-14 11:43:00,Kris Jenner: You're Cruel To Khloe Kardashian ...,"Hooray for for forcefully standing up to you,...",0
37626,2012-06-14 17:57:00,Robert Pattinson Speaks Out About 'Awkward' Co...,sat down this week with GQ where he revealed ...,0
37627,2012-06-01 19:49:00,Lamar Odom Is Shockingly A Likely Choice For U...,has been intensely training in hopes to make ...,0
37628,2012-06-15 09:29:00,Justin Bieber's 'Today Show' Concert Was His M...,"There’s Bieber Fever in , 18, performed at To...",0


In [ ]:
Filter(hollywoodlife)
hollywoodlife

,date,title,text,is_fake
0,2021-06-30 19:30:00,Bebe Rexha Proudly Rocks Lingerie To Promote B...,", 31, wants fans to know that she loves her bo...",0
1,2021-06-30 19:14:00,Jill Biden & Doug Emhoff Enjoy Beers Together ...,Cheers to baseball and COVID-19 vaccinations. ...,0
2,2021-07-01 00:23:00,Kevin Federline Denies Using His Kids As ‘Pawn...,‘s lawyer is making it clear he did not use hi...,0
3,2021-06-30 18:06:00,Prince Harry Admits It’s ‘A Juggle’ With 2 Kid...,Fellow British redheads and shared parenting ...,0
4,2021-06-30 19:12:00,Simone Biles Beats Boyfriend Jonathan Owens In...,"Not so fast! NFL player , 25, challenged girlf...",0
...,...,...,...,...
37498,2012-05-14 11:43:00,Kris Jenner: You're Cruel To Khloe Kardashian ...,"Hooray for for forcefully standing up to you,...",0
37499,2012-06-14 17:57:00,Robert Pattinson Speaks Out About 'Awkward' Co...,sat down this week with GQ where he revealed ...,0
37500,2012-06-01 19:49:00,Lamar Odom Is Shockingly A Likely Choice For U...,has been intensely training in hopes to make ...,0
37501,2012-06-15 09:29:00,Justin Bieber's 'Today Show' Concert Was His M...,"There’s Bieber Fever in , 18, performed at To...",0


In [ ]:
#Chuyển dữ liệu về kiểu string để so sánh
hollywoodlife = hollywoodlife.astype({"date": str})

In [ ]:
Delete_From_DataFrame(hollywoodlife)
hollywoodlife

,date,title,text,is_fake
0,2021-06-30 19:30:00,Bebe Rexha Proudly Rocks Lingerie To Promote B...,", 31, wants fans to know that she loves her bo...",0
1,2021-06-30 19:14:00,Jill Biden & Doug Emhoff Enjoy Beers Together ...,Cheers to baseball and COVID-19 vaccinations. ...,0
2,2021-07-01 00:23:00,Kevin Federline Denies Using His Kids As ‘Pawn...,‘s lawyer is making it clear he did not use hi...,0
3,2021-06-30 18:06:00,Prince Harry Admits It’s ‘A Juggle’ With 2 Kid...,Fellow British redheads and shared parenting ...,0
4,2021-06-30 19:12:00,Simone Biles Beats Boyfriend Jonathan Owens In...,"Not so fast! NFL player , 25, challenged girlf...",0
...,...,...,...,...
25889,2018-01-02 22:00:00,T.I. Totally Turned On By Tiny While Performin...,"., 37 and marriage is as steamy as ever. After...",0
25890,2018-01-02 09:32:00,Does Tamar Braxton Have A Side Piece Of Her Ow...,This is insane! An anonymous woman contacted ...,0
25891,2018-01-01 15:04:00,Tiffany Trump Shows Off Cleavage & Long Legs I...,"Ooh la la!, 24, stunned when she welcomed the ...",0
25892,2018-01-02 12:14:00,Lauren Jauregui & Ty Dolla Sign Kiss On NYE & ...,", 21, and , 32, are going strong, and the proo...",0


##Wtop

In [ ]:
import scrapy

class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://wtop.com/national/',
            'https://wtop.com/sports/',
            'https://wtop.com/business-finance/recalls/',
            'https://wtop.com/business-finance/washington-business-journal/',
            'https://wtop.com/business-finance/consumer-news/',
            'https://wtop.com/business-finance/real-estate/',
            'https://wtop.com/world/australia/',
            'https://wtop.com/world/africa/',
            'https://wtop.com/world/canada/',
            'https://wtop.com/world/europe/',
            'https://wtop.com/world/latin-america/',
            'https://wtop.com/world/middle-east/',
            'https://wtop.com/lifestyle/health-fitness/coronavirus/',
            'https://wtop.com/lifestyle/health-fitness/coronavirus/',
            'https://wtop.com/local/maryland/',
            'https://wtop.com/local/virginia/',
            'https://wtop.com/local/crime/',
            'https://wtop.com/local/beach-guide/',
            'https://wtop.com/local/local-politics-elections-news/',
            'https://wtop.com/dc-transit/',
            'https://wtop.com/weather-news/',
            'https://wtop.com/local/maryland/anne-arundel-county/',
            'https://wtop.com/local/maryland/baltimore/',
            'https://wtop.com/local/maryland/calvert-county/',
            'https://wtop.com/local/maryland/charles-county/',
            'https://wtop.com/local/maryland/frederick-county/',
            'https://wtop.com/local/maryland/howard-county/',
            'https://wtop.com/local/maryland/montgomery-county/',
            'https://wtop.com/local/maryland/prince-georges-county/',
            'https://wtop.com/government/white-house/',
            'https://wtop.com/government/supreme-court/',
            'https://wtop.com/government/congress/',
            'https://wtop.com/entertainment/arts/',
            'https://wtop.com/entertainment/celebrities/',
            'https://wtop.com/entertainment/movies/',
            'https://wtop.com/entertainment/music/',
            'https://wtop.com/entertainment/tv/'
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        getLinkInDiv = response.css('h3.post__template-title')
        allLinks = getLinkInDiv.css('a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)
        next_page = response.css('a.next::attr(href)')[-1].get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']  
        available = False  
        date = response.css('p.article-post__date::text').get() 
        for year in yearsList:  
            if year in date:  
                available = True  
                break
        if available:
            title = response.css('.page__single--title::text').get()
            paragraph = response.css('.entry-content p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 3:
                    break
                if count == 0:
                    count += 1
                    continue
                sentence.replace('\xa0', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 0
            }

In [ ]:
wtop = pd.read_json('https://raw.githubusercontent.com/nguyenthithaohien/data/main/data20.json')
Filter(wtop)
wtop

,date,title,text,is_fake
0,2021-07-05 15:12:00,Vatican: Pope alert and well a day after intes...,"Francis, 84, is expected to stay in Rome’s Gem...",0
1,2021-07-05 23:26:00,Maroon 3-peat? Lightning forward can join elit...,"After accomplishing that in 2019, he signed wi...",0
2,2021-07-05 23:34:00,"Gamel hits 2 HRs, drives in 6 runs; Pirates ri...",Maybe they’ll stick in Pittsburgh. Maybe they ...,0
3,2021-07-05 11:37:00,Census takers worry that apartment renters wer...,"“I had a few landlords who said, ‘It’s COVID. ...",0
4,2021-07-05 15:03:00,"Journalists attacked, hurt in Georgia at anti-...",Organizers of the Tbilisi March For Dignity th...,0
...,...,...,...,...
28067,2020-10-30 16:29:00,Trump’s election night party up in air due to ...,Trump told reporters that he may stay at the W...,0
28068,2020-10-31 16:30:00,"Obama criticizes Trump in scathing, personal t...","Campaigning for Joe Biden on Saturday, the for...",0
28069,2020-10-30 00:03:00,Trump tests limits as Cabinet members fan out ...,That was just Thursday.Members of President Do...,0
28070,2020-10-08 10:54:00,DC-area health officials to White House staff:...,“Given the growing numbers of positive COVID c...,0


##Buzzfeed

In [ ]:
import scrapy
offset = 1
valid = True


class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://www.buzzfeednews.com/'
        ]

        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        global offset

        allLinks = response.css('a.newsblock-story-card__link::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = 'https://www.buzzfeednews.com/us/feed/home?page='
        offset += 1
        next_page = next_page + str(offset) + '&flexpro_enabled=1'
        next_page = response.urljoin(next_page)
        yield scrapy.Request(next_page, callback=self.parse, dont_filter=True)

    def parse_href(self, response):
        global valid
        yearsList = ['2021', '2020', '2019', '2018']
        available = False
        date = response.css('.news-article-header__timestamps-posted::text').get()
        if date is not None:
            date = date.replace('\n    Posted on ', '')
            temp = date.split(', at', 1)
            date = temp[0]
            for year in yearsList:
                if year in date:
                    available = True
                    break
        if available:
            title = response.css('.news-article-header h1::text').get()
            title = title.replace('\\', '')
            paragraph = response.css('.subbuzz p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 3:
                    break
                sentence = sentence.replace('\xa0', '')
                sentence = sentence.replace('\n', '')
                sentence = sentence.replace('\r', '')
                sentence = sentence.replace('\t', '')
                sentence = sentence.replace('\'', ' ')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 0
            }


In [ ]:
buzz = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/buzzfeed.json')
Filter(buzz)
buzz

,date,title,text,is_fake
0,2021-07-06,Prosecutors Have Had Six Months To Go After Th...,Trump supporters clash with police and securit...,0
1,2021-07-01,A Former Cop Charged In The Capitol Attack Has...,Jacob Fracker (left) and Thomas Robertson insi...,0
2,2021-07-01,The Trump Organization And A Longtime Executiv...,Allen Weisselberg at the hearing for the case ...,0
3,2021-07-01,Influencers In Norway Will Legally Have To Dis...,Legislators in Norway have announced that wil...,0
4,2021-07-01,Track The Brutal 2021 Wildfire Season With The...,"The West is a tinderbox this year, with heat w...",0
...,...,...,...,...
195,2021-07-06,"Jennifer Lopez Revealed She's ""Super Happy"" An...","""I love what I m doing, I love where I m at, I...",0
196,2021-07-06,How Autistic People Are Showing The Limitation...,"While speaking at an event, autistic advocate ...",0
197,2021-07-06,Eric Adams Has Won New York City's Democratic ...,Brooklyn Borough president and mayoral candida...,0
198,2021-07-06,"They’re Vaccinated, But They Still Can’t Trave...",Left: Nadja De Maeseneer and her husband Juliu...,0


##Thesun

In [ ]:
import scrapy


class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://www.thesun.co.uk/news/uknews/',
            'https://www.thesun.co.uk/news/worldnews/',
            'https://www.thesun.co.uk/news/brexit/',
            'https://www.thesun.co.uk/news/politics/',
            'https://www.thesun.co.uk/news/health-news/',
            'https://www.thesun.co.uk/tech/science/',
            'https://www.thesun.co.uk/tech/phones-gadgets/',
            'https://www.thesun.co.uk/tech/gaming/',
            'https://www.thesun.co.uk/money/news-money/',
            'https://www.thesun.co.uk/money/shopping/',
            'https://www.thesun.co.uk/money/business/'
        ]

        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):

        allLinks = response.css('a.teaser-anchor::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = response.css('a.pagination-next::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']
        available = False
        date = response.css('.article__timestamp::text').get()
        if date is not None:
            for year in yearsList:
                if year in date:
                    available = True
                    break
        if available:
            title = response.css('.article__headline::text').get()
            paragraph = response.css('.article__content p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 3:
                    break
                sentence = sentence.replace('\xa0', '')
                sentence = sentence.replace('\n', '')
                sentence = sentence.replace('\r', '')
                sentence = sentence.replace('\t', '')
                sentence = sentence.replace('\'', ' ')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 0
            }


File's link: https://drive.google.com/file/d/1LF-Nl107wEvqcnJ4JXv0ayA5jx0zLm53/view?usp=sharing

In [ ]:
sun = pd.read_json('drive/MyDrive/thesun.json')
Filter(sun)
sun

,date,title,text,is_fake
0,2021-07-05,Ordering at the bar could still be banned AFTE...,ORDERING at the bar could still be banned afte...,0
1,2021-07-05,Boris Johnson’s plan to scrap masks after July...,BORIS Johnson s plan to axe masks on buses and...,0
2,2021-07-05,Over 3million manual workers need to retrain t...,MORE than three ­million manual workers will h...,0
3,2021-07-05,Ordering pints at the bar to return in two wee...,DRINKERS will be back standing at the bar orde...,0
4,2021-07-05,What is Boris Johnson’s full name?,"BORIS Johnson is a household name, known acros...",0
...,...,...,...,...
99029,2020-11-27,Cop could be sacked for ‘sticking photo of fem...,Sgt Rob Adams is said to have covered the nake...,0
99030,2020-11-28,Back our blanket appeal and help premature bab...,"Last December, twins Harper and Poppy were in ...",0
99031,2018-09-19,Baby giraffe whose protective mother attacked ...,THE baby giraffe caught up in the horrific att...,0
99032,2018-09-18,Brit woman Asa Hutchinson given Dubai prison s...,THE British expat slapped with a prison senten...,0


##Wsvn

In [ ]:
import scrapy

class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://wsvn.com/news/'
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        getLinkInDiv = response.css('h3.archive-item__title')
        allLinks = getLinkInDiv.css('a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)
        next_page = response.css('a.next::attr(href)')[-1].get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']  # Tạo ra 1 cái list chứa các năm được chấp thuận
        available = False
        date = response.css('div.article-header__meta time::text').get()
        for year in yearsList:  # Duyệt qua mỗi năm
            if year in date:  # Nếu trong bài viết có chứa cái năm được chấp thuận
                available = True  # Biến = true
                break
        if available:
            title = response.css('.article-header__title::text').get()
            paragraph = response.css('.article-body p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 3:
                    break
                if count == 0:
                    count += 1
                    continue
                sentence.replace('\xa0', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 0
            }

In [ ]:
wsvn = pd.read_json('https://raw.githubusercontent.com/nguyenthithaohien/data/main/data25.json')
wsvn  = wsvn.reset_index(drop=True).rename(columns = {'i':'is_fake'})
Filter(wsvn)
wsvn

,date,title,text,is_fake
0,2021-07-06,"Funerals held for family of 4, beloved matriar...",Three caskets were carried into St. Joseph’s C...,0
1,2021-07-06,Woman released from hospital after monthslong ...,Donna Whedbee was brought to tears Tuesday as ...,0
2,2021-07-06,Officials allow for up-close view of ongoing s...,The condo building is where people lived and c...,0
3,2021-07-06,Officials: Storm lashing Florida strengthens i...,The National Weather Service said Tuesday that...,0
4,2021-07-06,Police search for woman missing from Margate,Christine Miller was last seen leaving her hom...,0
...,...,...,...,...
42693,2018-01-01,Detectives: Florida man beat 6-year-old stepso...,"Jack Junior Montgomery, 31, has been charged w...",0
42694,2018-01-01,1 dead after shooting at New Year’s Eve party ...,Officers responded to the home near Northwest ...,0
42695,2018-01-01,Prosecutor: Teen held in shooting death of par...,Monmouth County Prosecutor Chris Gramiccioni s...,0
42696,2018-01-01,Man hospitalized due to firework injury in NW ...,"The victim, identified as 25-year-old Orlando ...",0


#Tổng hợp trang tin chính thống

In [ ]:
data_real_news = eco.append(apnews, ignore_index=True)
real_news_websites = [npr, brg, india, slate, people, hollywoodlife, wtop, buzz, sun, wsvn]
for item in real_news_websites:
  data_real_news = data_real_news.append(item, ignore_index=True)
data_real_news

,date,title,text,is_fake
0,2021-06-24 00:00:00,We’re hiring: a news assistant in Tokyo,"is seeking a . This is an exciting, multiface...",0
1,2021-06-24 00:00:00,An important census product may soon use synth...,"(), which is sent to around 1% of America’s p...",0
2,2021-06-24 00:00:00,Myanmar’s civil war is becoming bloodier and m...,"bicycle, wearing a -shirt emblazoned with a c...",0
3,2021-06-24 00:00:00,America’s Supreme Court is less one-sided than...,America’s Supreme Court seemed destined for a...,0
4,2021-06-12 00:00:00,The anti-graft unit of China’s Communist Party...,"June 1st Shi Zhaoqing, a local boss in China’...",0
...,...,...,...,...
353371,2018-01-01 00:00:00,Detectives: Florida man beat 6-year-old stepso...,"Jack Junior Montgomery, 31, has been charged w...",0
353372,2018-01-01 00:00:00,1 dead after shooting at New Year’s Eve party ...,Officers responded to the home near Northwest ...,0
353373,2018-01-01 00:00:00,Prosecutor: Teen held in shooting death of par...,Monmouth County Prosecutor Chris Gramiccioni s...,0
353374,2018-01-01 00:00:00,Man hospitalized due to firework injury in NW ...,"The victim, identified as 25-year-old Orlando ...",0


#Chuyển dữ liệu đã thu thập được thành file.json 

Tham khảo tại: https://datatofish.com/export-pandas-dataframe-json/

In [ ]:
from google.colab import files
data_fake_news = data_fake_news.astype({"date":str})
data_fake_news.to_json('data_fake_news.json', orient='records')
files.download('data_fake_news.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
data_real_news = data_real_news.astype({"date":str})
data_real_news.to_json('data_real_news.json', orient='records')
files.download('data_real_news.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>